In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.utils import shuffle
import pandas as pd
import pickle
from matplotlib.pyplot import MultipleLocator

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
"""读入数据"""
train_data = pd.read_csv(r'../input/blood-pressure-datasets/Train_Merge_Data.csv')
train_dev = pd.read_csv(r'../input/ppg-dev-new/train.csv')

validation_data = pd.read_csv(r'../input/blood-pressure-datasets/Validation_Merge_Data.csv')
validation_dev = pd.read_csv(r'../input/ppg-dev-new/validation.csv')

In [ ]:
"""组合训练数据和一阶导、二阶导数据"""
train_data = pd.concat([train_data,train_dev],axis=1)
validation_data = pd.concat([validation_data,validation_dev],axis=1)

In [ ]:
"""打乱数据"""
train_data = shuffle(train_data)
validation_data = shuffle(validation_data)

In [ ]:
"""加载训练集"""
train_label_Sbp = train_data.iloc[:,610]
train_label_Sbp = train_label_Sbp.values

train_label_Dbp = train_data.iloc[:,611]
train_label_Dbp = train_label_Dbp.values

train_dev1 = train_data.iloc[:,662:1262]
train_dev1 = train_dev1.values

train_dev2 = train_data.iloc[:,1262:]
train_dev2 = train_dev2.values

train_data = train_data.iloc[:,:600]
train_data = train_data.values


In [ ]:
"""加载验证集"""
validation_label_Sbp = validation_data.iloc[:,610]
validation_label_Sbp = validation_label_Sbp.values

validation_label_Dbp = validation_data.iloc[:,611]
validation_label_Dbp = validation_label_Dbp.values

validation_dev1 = validation_data.iloc[:,662:1262]
validation_dev1 = validation_dev1.values

validation_dev2 = validation_data.iloc[:,1262:]
validation_dev2 = validation_dev2.values

validation_data = validation_data.iloc[:,:600]
validation_data = validation_data.values

In [ ]:
"""数据格式、尺寸"""
print("train_data_information:")
print(train_data.shape)
print(train_dev1.shape)
print(train_dev2.shape)
print(train_label_Sbp.shape)
print(train_label_Dbp.shape)

print("validation_data_information:")
print(validation_data.shape)
print(validation_dev1.shape)
print(validation_dev2.shape)
print(validation_label_Sbp.shape)
print(validation_label_Dbp.shape)

In [ ]:
"""注意力机制模块"""
#---------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------
# 注意力机制：Relu6方式
# 激活函数 relu6
def relu6(x):
    return tf.keras.activations.relu(x, max_value=6)
#   利用relu函数乘上x模拟sigmoid
def hard_swish(x):
    return x * tf.keras.activations.relu(x + 3.0, max_value=6.0) / 6.0

#---------------------------------------#
#   通道注意力机制单元
#   利用两次全连接算出每个通道的比重
#   可以连接在任意特征层后面
#---------------------------------------#

def squeeze(inputs):
    input_channels = int(inputs.shape[-1])
    
    x = layers.GlobalAveragePooling1D()(inputs)

    x = layers.Dense(int(input_channels/4))(x)
    x = relu6(x)

    x = layers.Dense(input_channels)(x)
    x = hard_swish(x)

    x = layers.Reshape((1, input_channels))(x)
    #print(x)
    #print(inputs)
    x = layers.Multiply()([inputs, x])
    return x

#---------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------
#注意力机制：Sigmoid方式
# def squeeze(inputs):
#     input_channels = int(inputs.shape[-1])
    
#     x = layers.GlobalAveragePooling1D()(inputs)

#     x = layers.Dense(input_channels//4)(x)
#     x = layers.Activation('relu')(x)

#     x = layers.Dense(input_channels)(x)
#     x = layers.Activation('softmax')(x)

#     x = layers.Reshape((1, input_channels))(x)
#     #print(x)
#     #print(inputs)
#     x = layers.Multiply()([inputs, x])
#     return x
#---------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------

In [ ]:
"""下采样"""
#---------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------
#基础层—3x3_3x3—下采样——改变卷积层步长的方式
#ps:目前试验结果中最好的下采样方式
def DownSampling(inputs,filters,stride_padding='same',Drop=False):
    layer = layers.Conv1D(filters,3,padding='same')(inputs)
    layer = layers.BatchNormalization()(layer)
    layer = layers.Activation(tf.nn.relu)(layer)

    layer = layers.Conv1D(filters,3,padding='same')(layer)
    layer = layers.BatchNormalization()(layer)
    
    res_layer = layers.Conv1D(filters,1,padding='same')(inputs)
    res_layer = layers.BatchNormalization()(res_layer)
    layer = layers.add([layer,res_layer])
    
    layer = layers.Activation(tf.nn.relu)(layer)
    
    if Drop==True:
        layer = layers.Dropout(0.25)(layer)
    
    pool_inputs = layers.Conv1D(filters,3,padding=stride_padding,strides=2)(layer)
    pool = layers.BatchNormalization()(pool_inputs)
    pool = layers.Activation(tf.nn.relu)(pool)

    pool = layers.Conv1D(filters,3,padding='same')(pool)
    pool = layers.BatchNormalization()(pool)
    
    res_pool = layers.Conv1D(filters,1,padding='same')(pool_inputs)
    res_pool = layers.BatchNormalization()(res_pool)
    pool = layers.add([pool,res_pool])
    
    pool = layers.Activation(tf.nn.relu)(pool)
    
    
    return layer,pool
# ---------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------
#基础层—3x3_3x3—下采样——池化的方式
# def DownSampling(inputs,filters,stride_padding='same',Drop=False):
#     layer = layers.Conv1D(filters,3,padding='same')(inputs)
#     layer = layers.BatchNormalization()(layer)
#     layer = layers.Activation(tf.nn.relu)(layer)

#     layer = layers.Conv1D(filters,3,padding='same')(layer)
#     layer = layers.BatchNormalization()(layer)
    
#     res_layer = layers.Conv1D(filters,1,padding='same')(inputs)
#     res_layer = layers.BatchNormalization()(res_layer)
#     layer = layers.add([layer,res_layer])
    
#     layer_inputs = layers.Activation(tf.nn.relu)(layer)

#     layer = layers.Conv1D(filters,3,padding=stride_padding,strides=1)(layer_inputs)
#     layer = layers.BatchNormalization()(layer)
#     layer = layers.Activation(tf.nn.relu)(layer)

#     layer = layers.Conv1D(filters,3,padding='same')(layer)
#     layer = layers.BatchNormalization()(layer)
    
#     res_layer = layers.Conv1D(filters,1,padding='same')(layer_inputs)
#     res_layer = layers.BatchNormalization()(res_layer)
#     layer = layers.add([layer,res_layer])
#     layer = layers.Activation(tf.nn.relu)(layer)
    
#     if Drop==True:
#         layer = layers.Dropout(0.25)(layer)
    
#     #平均池化
#     pool = layers.AveragePooling1D(pool_size=2)(layer)
#     #最大池化
#     #pool = layers.MaxPooling1D(pool_size=2)(layer)
#     return layer,pool
#---------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------
# 每层单个残差块
# def DownSampling(inputs,filters,stride_padding='same',Drop=False):
#     layer = layers.Conv1D(filters,3,padding=stride_padding)(inputs)
#     layer = layers.BatchNormalization()(layer)
#     layer = layers.Activation(tf.nn.relu)(layer)

#     layer = layers.Conv1D(filters,3,padding='same')(layer)
#     layer = layers.BatchNormalization()(layer)
    
#     res_layer = layers.Conv1D(filters,1,padding='same')(inputs)
#     res_layer = layers.BatchNormalization()(res_layer)
#     layer = layers.add([layer,res_layer])
    
#     layer = layers.Activation(tf.nn.relu)(layer)
    
#     if Drop==True:
#         layer = layers.Dropout(0.25)(layer)
    
#     #平均池化
#     pool = layers.AveragePooling1D(pool_size=2)(layer)
#     #最大池化
#     #pool = layers.MaxPooling1D(pool_size=2)(layer)
#     return layer,pool


In [ ]:
"""上采样"""
#基础层上采样
def UpSampling(inputs,con_input,filters,need_zero=False):
    
    
    up_layer = layers.UpSampling1D(size=2)(inputs)
    
    if need_zero==True:
        up_layer = layers.ZeroPadding1D((0,1))(up_layer)
    
    layer = layers.Conv1D(filters,2,padding='same')(up_layer)
    layer = layers.BatchNormalization()(layer)
    layer = layers.Activation(tf.nn.relu)(layer)
    
    layer = layers.Concatenate(axis=2)([layer,con_input])
    
    layer = layers.Conv1D(filters,3,padding='same')(layer)
    layer = layers.BatchNormalization()(layer)
    layer = layers.Activation(tf.nn.relu)(layer)

    layer = layers.Conv1D(filters,3,padding='same')(layer)
    layer = layers.BatchNormalization()(layer)
    output = layers.Activation(tf.nn.relu)(layer)
    
    return output

In [ ]:
"""上采样结束的卷积操作"""
def FinalConv1D(inputs,filters):
    layer = layers.Conv1D(filters,3,padding='same')(inputs)
    layer = layers.BatchNormalization()(layer)
    layer = layers.Activation(tf.nn.relu)(layer)
    
    layer = layers.MaxPooling1D(pool_size=2)(layer)
    
    layer = layers.Conv1D(filters,3,padding='same')(layer)
    layer = layers.BatchNormalization()(layer)
    layer = layers.Activation(tf.nn.relu)(layer)
    
    layer = layers.MaxPooling1D(pool_size=2)(layer)
    return layer

In [ ]:
# def create_model():
#     #双输入模型
#     inputs_1 = keras.Input(shape=(600,1),name='inputs_1')
#     inputs_dev1 = keras.Input(shape=(600,1),name='inputs_dev1')
#     inputs_dev2 = keras.Input(shape=(600,1),name='inputs_dev2')
    
#     inputs = layers.Concatenate(axis=2)([inputs_1,inputs_dev1,inputs_dev2])
    
#     #下采样
#     layer1,pool1 = DownSampling(inputs,filters=64)
    
#     layer2,pool2 = DownSampling(pool1,filters=128)
        
#     layer3,pool3 = DownSampling(pool2,filters=256)
    
#     layer4,pool4 = DownSampling(pool3,filters=512,stride_padding='valid',Drop=False)
    
#     #最后一层不需要Pooling
#     layer51,pool51 = DownSampling(pool4,filters=1024,Drop=False)
#     layer52,pool52 = DownSampling(pool4,filters=1024,Drop=False)
#     layer53,pool53 = DownSampling(pool4,filters=1024,Drop=False)
    
#     layer51 = squeeze(layer51)
#     layer52 = squeeze(layer52)
#     layer53 = squeeze(layer53)
    
#     layer51 = layers.BatchNormalization()(layer51)
#     layer52 = layers.BatchNormalization()(layer52)
#     layer53 = layers.BatchNormalization()(layer53)
#     layer5 = layers.add([layer51,layer52])
#     layer5 = layers.add([layer5,layer53])
    
#     #layer5 = layers.BatchNormalization()(layer5)
    
#     #上采样
    
#     #attention
#     layer4 = squeeze(layer4)
    
#     layer44 = UpSampling(layer5,layer4,512,need_zero=True)
    
#     #attention
#     layer3 = squeeze(layer3)
                     
#     layer33 = UpSampling(layer44,layer3,256)
    
#     #attention
#     layer2 = squeeze(layer2)
    
#     layer22 = UpSampling(layer33,layer2,128)
    
#     #attention
#     layer1_1 = squeeze(layer1)
    
#     layer1_2 = squeeze(layer1)
    
#     layer1_3 = squeeze(layer1)
    
#     layer111 = UpSampling(layer22,layer1_1,64)
    
#     layer112 = UpSampling(layer22,layer1_2,64)
    
#     layer113 = UpSampling(layer22,layer1_3,64)
    
#     layer111 = FinalConv1D(layer111,filters=64)
#     layer112 = FinalConv1D(layer112,filters=64)
#     layer113 = FinalConv1D(layer113,filters=64)
    
#     layer1111 = squeeze(layer111)
    
#     layer1112 = squeeze(layer112)
    
#     layer1113 = squeeze(layer113)
    
    
#     layer1111 = layers.BatchNormalization()(layer1111)
#     layer1112 = layers.BatchNormalization()(layer1112)
#     layer1113 = layers.BatchNormalization()(layer1113)
#     layer1111 = layers.add([layer1111,layer1112])
#     layer11 = layers.add([layer1111,layer1113])
    
#     layer = layers.GlobalAveragePooling1D()(layer11)

#     outputs_sbp = layers.Dense(1,name='Sbp')(layer)
#     outputs_dbp = layers.Dense(1,name='Dbp')(layer)

#     model = keras.Model(inputs=[inputs_1,inputs_dev1,inputs_dev2],outputs=[outputs_sbp,outputs_dbp])
    
#     return model

In [ ]:
# #ARU-Net原版
# def create_model():
#     #双输入模型
#     inputs_1 = keras.Input(shape=(600,1),name='inputs_1')
#     inputs_dev1 = keras.Input(shape=(600,1),name='inputs_dev1')
#     inputs_dev2 = keras.Input(shape=(600,1),name='inputs_dev2')
    
#     inputs = layers.Concatenate(axis=2)([inputs_1,inputs_dev1,inputs_dev2])
    
#     #下采样
#     layer1,pool1 = DownSampling(inputs,filters=64)
    
#     layer2,pool2 = DownSampling(pool1,filters=128)
        
#     layer3,pool3 = DownSampling(pool2,filters=256)
    
#     layer4,pool4 = DownSampling(pool3,filters=512,stride_padding='valid',Drop=False)
    
#     #最后一层不需要Pooling
#     layer5,pool5 = DownSampling(pool4,filters=1024,Drop=False)
    
#     #上采样
    
#     #attention
#     layer4 = squeeze(layer4)
    
#     layer44 = UpSampling(layer5,layer4,512,need_zero=True)
    
#     #attention
#     layer3 = squeeze(layer3)
                     
#     layer33 = UpSampling(layer44,layer3,256)
    
#     #attention
#     layer2 = squeeze(layer2)
    
#     layer22 = UpSampling(layer33,layer2,128)
    
#     #attention
#     layer1 = squeeze(layer1)
    
#     layer11 = UpSampling(layer22,layer1,64)
    
    
#     layer = layers.GlobalAveragePooling1D()(layer11)

#     outputs_sbp = layers.Dense(1,name='Sbp')(layer)
#     outputs_dbp = layers.Dense(1,name='Dbp')(layer)

#     model = keras.Model(inputs=[inputs_1,inputs_dev1,inputs_dev2],outputs=[outputs_sbp,outputs_dbp])
    
#     return model

In [ ]:
#W-Net
def create_model():
    #双输入模型
    inputs_1 = keras.Input(shape=(600,1),name='inputs_1')
    inputs_dev1 = keras.Input(shape=(600,1),name='inputs_dev1')
    inputs_dev2 = keras.Input(shape=(600,1),name='inputs_dev2')
    
    inputs = layers.Concatenate(axis=2)([inputs_1,inputs_dev1,inputs_dev2])
    #11111111111111111111111111111111111111111111111
    #下采样
    layer_1_1,pool_1_1 = DownSampling(inputs,filters=64)
    
    layer_1_2,pool_1_2 = DownSampling(pool_1_1,filters=128)
        
    layer_1_3,pool_1_3 = DownSampling(pool_1_2,filters=256)
    
    layer_1_4,pool_1_4 = DownSampling(pool_1_3,filters=512,stride_padding='valid',Drop=False)
    
    #最后一层不需要Pooling
    layer_1_5,pool_1_5 = DownSampling(pool_1_4,filters=1024,Drop=False)
    
    #上采样
    
    #attention
    layer_1_4 = squeeze(layer_1_4)
    
    layer_1_44 = UpSampling(layer_1_5,layer_1_4,512,need_zero=True)
    
    #attention
    layer_1_3 = squeeze(layer_1_3)
                     
    layer_1_33 = UpSampling(layer_1_44,layer_1_3,256)
    
    #attention
    layer_1_2 = squeeze(layer_1_2)
    
    layer_1_22 = UpSampling(layer_1_33,layer_1_2,128)
    
    #attention
    layer_1_1 = squeeze(layer_1_1)
    
    layer_1_11 = UpSampling(layer_1_22,layer_1_1,64)
    
    layer_dbp = layers.GlobalAveragePooling1D()(layer_1_11)
    
    
    #222222222222222222222222222222222222222222222
     #下采样
    layer_2_1,pool_2_1 = DownSampling(layer_1_11,filters=64)
    
    layer_2_2,pool_2_2 = DownSampling(pool_2_1,filters=128)
        
    layer_2_3,pool_2_3 = DownSampling(pool_2_2,filters=256)
    
    layer_2_4,pool_2_4 = DownSampling(pool_2_3,filters=512,stride_padding='valid',Drop=False)
    
    #最后一层不需要Pooling
    layer_2_5,pool_2_5 = DownSampling(pool_2_4,filters=1024,Drop=False)
    
    #上采样
    
    #attention
    layer_2_4 = squeeze(layer_2_4)
    
    layer_2_44 = UpSampling(layer_2_5,layer_2_4,512,need_zero=True)
    
    #attention
    layer_2_3 = squeeze(layer_2_3)
                     
    layer_2_33 = UpSampling(layer_2_44,layer_2_3,256)
    
    #attention
    layer_2_2 = squeeze(layer_2_2)
    
    layer_2_22 = UpSampling(layer_2_33,layer_2_2,128)
    
    #attention
    layer_2_1 = squeeze(layer_2_1)
    
    layer_2_11 = UpSampling(layer_2_22,layer_2_1,64)
    
    
    
    
    
    
    layer_sbp = layers.GlobalAveragePooling1D()(layer_2_11)

    outputs_sbp = layers.Dense(1,name='Sbp')(layer_sbp)
    outputs_dbp = layers.Dense(1,name='Dbp')(layer_dbp)

    model = keras.Model(inputs=[inputs_1,inputs_dev1,inputs_dev2],outputs=[outputs_sbp,outputs_dbp])
    
    return model

In [ ]:
"""自定义评价指标模块"""
def standard_deviation(y_true, y_pred):
    u = keras.backend.mean(y_pred-y_true)
    return keras.backend.sqrt(keras.backend.mean(keras.backend.square((y_pred-y_true) - u)))

In [ ]:
"""回调函数"""
#保存迭代周期内最好的模型
checkpoint_filepath = r'./model_struction.h5'
Save_epochs = 100 #迭代多少层保存一次模型
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    # save_weights_only=True,
    monitor='val_Sbp_mean_absolute_error',
    mode='min',
    save_best_only=True)

In [ ]:
"""建立模型"""
with strategy.scope():
    model = create_model()
    
    model.compile(loss={'Sbp':"mse",'Dbp':"mse"}, optimizer=keras.optimizers.Adam(lr=0.0001),metrics=[tf.keras.metrics.MeanAbsoluteError(),standard_deviation])

In [ ]:
model.summary()

In [ ]:
"""保存模型结构图片"""
tf.keras.utils.plot_model(model, to_file=r'./model_graph.png', show_shapes=True, show_layer_names=False)


In [ ]:
history = model.fit({'inputs_1':train_data,'inputs_dev1':train_dev1,'inputs_dev2':train_dev2},{'Sbp':train_label_Sbp,'Dbp':train_label_Dbp},
                    batch_size=128*8,
                    epochs=500,
                    callbacks=model_checkpoint_callback,
                    validation_data=({'inputs_1':validation_data,'inputs_dev1':validation_dev1,'inputs_dev2':validation_dev2},{'Sbp':validation_label_Sbp,'Dbp':validation_label_Dbp})
                    )

In [ ]:
with open(r'./last_model.h5.pickle', 'wb') as file_pi:
 	pickle.dump(history.history, file_pi)

In [ ]:
history = model.fit({'inputs_1':train_data,'inputs_dev1':train_dev1,'inputs_dev2':train_dev2},{'Sbp':train_label_Sbp,'Dbp':train_label_Dbp},
                    batch_size=128*8,
                    epochs=500,
                    callbacks=model_checkpoint_callback,
                    validation_data=({'inputs_1':validation_data,'inputs_dev1':validation_dev1,'inputs_dev2':validation_dev2},{'Sbp':validation_label_Sbp,'Dbp':validation_label_Dbp})
                    )

In [ ]:
history = model.fit({'inputs_1':train_data,'inputs_dev1':train_dev1,'inputs_dev2':train_dev2},{'Sbp':train_label_Sbp,'Dbp':train_label_Dbp},
                    batch_size=128*8,
                    epochs=500,
                    callbacks=model_checkpoint_callback,
                    validation_data=({'inputs_1':validation_data,'inputs_dev1':validation_dev1,'inputs_dev2':validation_dev2},{'Sbp':validation_label_Sbp,'Dbp':validation_label_Dbp})
                    )